yolo는 dataset/images/train, dataset/images/valid 형식을 이용해야 함... <br/>
k fold (k = 5) json 기준으로 5개의 폴더 내에 train, valid 파일을 각각 복사하여 제작

In [ ]:
from sklearn.model_selection import train_test_split
import json
import shutil
from tqdm import tqdm

# json 파일에 들어있는 validation용 이미지를 이용하여 valid 폴더에 이미지를 복사합니다.
for j in range(5):
    with open('/opt/ml/dataset/kfold' + str(j) + '_train.json', 'r') as f:
        dataset = json.load(f)

    for i in tqdm(dataset['images']):
        shutil.copyfile('/opt/ml/dataset/images/' + i['file_name'], '/opt/ml/dataset/images_fold' + str(j) + '/train/' + i['file_name'][6:])
    
for j in range(5):
    with open('/opt/ml/dataset/kfold' + str(j) + '_val.json', 'r') as f:
        dataset = json.load(f)

    for i in tqdm(dataset['images']):
        shutil.copyfile('/opt/ml/dataset/images/' + i['file_name'], '/opt/ml/dataset/images_fold' + str(j) + '/valid/' + i['file_name'][6:])

yolo는 dataset/images/train, dataset/images/valid 형식을 이용해야 함... <br/>
random split으로 만들어진 json 기준으로 5개의 폴더 내에 train, valid 파일을 각각 복사하여 제작

In [ ]:
from sklearn.model_selection import train_test_split
import json
import shutil
from tqdm import tqdm

# json 파일에 들어있는 validation용 이미지를 이용하여 valid 폴더에 이미지를 복사합니다.
with open('/opt/ml/dataset/train_split.json', 'r') as f:
    dataset = json.load(f)

for i in tqdm(dataset['images']):
    shutil.copyfile('/opt/ml/dataset/images/' + i['file_name'], '/opt/ml/dataset/split_images/train/' + i['file_name'][6:])
    
with open('/opt/ml/dataset/val_split.json', 'r') as f:
    dataset = json.load(f)

for i in tqdm(dataset['images']):
    shutil.copyfile('/opt/ml/dataset/images/' + i['file_name'], '/opt/ml/dataset/split_images/valid/' + i['file_name'][6:])

https://github.com/Kim-jy0819/yolov8 깃허브 참조 <br/><br/>

dataset/labels 폴더에 이미지의 객체 정보를 저장하는 txt 파일이 존재해야 함... -> coco2yolo.py 폴더가 이를 해결함 <br/>
!python coco2yolo.py -j /opt/ml/dataset/train.json -o /opt/ml/dataset/labels/ <br/>
위 코드를 입력하면 kfold4_train.json 폴더의 내용을 기반으로 이미지의 annotation을 txt 파일로 생성하여 -o 뒤의 경로에 생성 <br/><br/>

단, json 파일은 기본적으로 train/0000.jpg의 이름을 가지고 있으므로 train 폴더가 반드시 생성됨 <br/>
따라서 train 내에 생성된 파일들을 valid 파일로 옮기거나, json의 값을 추출할때 train을 사용하지 않도록 coco2yolo.py를 수정해야 함

In [ ]:
!python coco2yolo.py -j /opt/ml/dataset/train.json -o /opt/ml/dataset/labels/

In [ ]:
# train.py에서 epoch, imgsz 등의 학습 설정 수정 가능
# ultralytics/yolo/cfg/default.yaml 에 기본적인 학습 정보가 존재
# ultralytics/yolo/engine/trainer.py에서 train loss 및 mAP metric 정보를 얻을 수 있음
# ultralytics/yolo/engine/validator.py에서 validation loss를 얻을 수 있음

# dataset/images, dataset/labels를 이용해야 하므로 학습을 위해 폴더 이름을 변경시켜둠
import os

os.rename('/opt/ml/dataset/split_labels', '/opt/ml/dataset/labels')
os.rename('/opt/ml/dataset/split_images', '/opt/ml/dataset/images')

!python train.py

os.rename('/opt/ml/dataset/labels', '/opt/ml/dataset/split_labels')
os.rename('/opt/ml/dataset/images', '/opt/ml/dataset/split_images')

In [ ]:
from ultralytics import YOLO
import wandb
import os

model = YOLO("/opt/ml/yolov8/runs/detect/train5/weights/last.pt")

model.train(data = "recycle.yaml",  # 데이터의 형식과 클래스의 이름
            # Train settings
            name = 'retrain', #결과를 저장할 폴더 이름
            epochs = 50,
            imgsz = 512, # 이미지 크기 조정
            pretrained = True,
            close_mosaic = 10,
            patience = 25, # 25 epoch동안 성능의 상승이 없으면 early stopping
            
            
            # Val/Test settings
            val = False, # validation 자체를 사용할지 말지에 대한 여부
            split = 'train', # [train,val, test]
            max_det = 100,  # maximum number of detections per image default: 300
            #iou = 0.7,  # intersection over union (IoU) threshold for NMS default:0.7
            
            # Hyperparameters
            fl_gamma = 1.5, # focal loss
            
            flipud = 0.33,
            fliplr = 0.33,
            mosaic = 1.0,
            
)  # train the model

In [ ]:
# 모델 경로를 넣고 제출용 csv를 만드는 용도의 inference
!python inference.py --model /opt/ml/yolov8/runs/detect/retrain4/weights/best.pt